# Переменные, встречающиеся в n разных папках.

In [1]:
cd ../

/home/ruslan/Documents/course-project-TiMP


Чтобы каждый раз не вводить путь до var.data.log файлов, которые генерируют пути AST для каждой папки, будем собирать их автоматически

In [2]:
import os, fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

files_path = find("*.var.data.log", "dataset/java-small")

In [3]:
def get_folder_name(path:str):
    begin_idx = path.rfind('/') + 1
    end_idx = path.rfind(".var.data.log")
    return path[begin_idx:end_idx]

Для каждого файла необходимо собрать его переменные (первое слово в строке) в отдельную таблицу

In [4]:
import pandas as pd
from collections import Counter

In [14]:
df = []
for file_path in files_path:
    df.append(pd.read_csv(file_path, sep=" ", usecols=[0], names = ["Variable"], header = None))
    df[-1]["Frequency"] = 1
    df[-1] = df[-1].groupby("Variable").sum().reset_index()
    df[-1]["Folders"] = 1

In [15]:
conc = pd.concat(df)
display(conc)

,Variable,Frequency,Folders
0,a,46,1
1,aborted,1,1
2,absolute,1,1
3,absolute|file,2,1
4,absolute|path,10,1
...,...,...,...
10938,zero|terms|query,6,1
10939,zip,4,1
10940,zip|input,1,1
10941,zone,2,1


In [25]:
conc = conc.groupby(["Variable"]).sum().reset_index()
conc = conc.query('Folders > 1 & Frequency > 50')
display(conc)
dict(zip(conc["Variable"].tolist(), conc["Frequency"].tolist()))
# conc = conc[conc["Frequency"] > 100]
# conc = conc[conc["Folders number"] > 2]
# conc.to_csv("freq.csv")
# display(conc)

,Variable,Frequency,Folders
0,a,2266,9
1,absolute|path,79,7
2,ac,68,6
3,access,182,9
4,accessor,341,5
...,...,...,...
1708,y,1459,9
1709,year,113,7
1710,z,59,6
1711,zip,91,7


{'a': 2266,
 'absolute|path': 79,
 'ac': 68,
 'access': 182,
 'accessor': 341,
 'access|token': 76,
 'account': 125,
 'account|id': 71,
 'action': 2007,
 'actions': 398,
 'action|id': 1131,
 'action|ids': 154,
 'action|mapping': 83,
 'action|name': 94,
 'action|request': 753,
 'action|response': 442,
 'active': 909,
 'activity': 137,
 'actual': 937,
 'actual|type': 51,
 'ad': 69,
 'adapter': 178,
 'add': 75,
 'added': 130,
 'addr': 117,
 'address': 1429,
 'addresses': 160,
 'address|id': 71,
 'advice': 53,
 'ae': 56,
 'after': 223,
 'age': 74,
 'aggregation': 92,
 'aggregations': 85,
 'algorithm': 83,
 'alias': 538,
 'aliases': 172,
 'all': 209,
 'allocation': 171,
 'analyzer': 188,
 'ancestor': 81,
 'anchor': 652,
 'ann': 129,
 'anno': 55,
 'annotation': 941,
 'annotations': 227,
 'annotation|class': 74,
 'annotation|name': 79,
 'annotation|type': 183,
 'anonymous': 72,
 'another': 100,
 'answer': 192,
 'an|object': 86,
 'app': 199,
 'application': 204,
 'application|context': 80,
 'a

In [8]:
freqs = conc['Frequency']
words = conc['Variable']

display(words, freqs)

4                a
254         access
266       accessor
342        account
430         action
            ...   
115421           x
115490         xml
115557    xml|file
115896           y
115903        year
Name: Variable, Length: 969, dtype: object

4         2266
254        182
266        341
342        125
430       2007
          ... 
115421    2308
115490     395
115557     160
115896    1459
115903     113
Name: Frequency, Length: 969, dtype: int64

In [9]:
cnt = Counter()
for seq, freq in zip(words, freqs):
    cnt.update({seq: freq})
    
LIMIT_WORDS=200
srt = sorted(cnt.items(), key=lambda x: x[1], reverse=True)
srt_labels, srt_freqs = list(map(list, zip(*srt)))
srt_labels = srt_labels[:LIMIT_WORDS] + ['other words']
srt_freqs = srt_freqs[:LIMIT_WORDS] + [sum(srt_freqs[LIMIT_WORDS:])]

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.figure(figsize=(15, 8))
plt.title(f"Распределение первых {LIMIT_WORDS} слов по встречаемости")
plt.yscale("log")
plt.xticks(rotation='vertical')
plt.margins(0.025)
plt.bar(srt_labels, srt_freqs)
plt.plot();

KeyboardInterrupt: 